<a href="https://colab.research.google.com/github/QuinteroR/CuTonala_2024_A/blob/RaulQM/CountSortParalelo_RaulQM_PrograParalela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from mpi4py import MPI
import numpy as np

def counting_sort_parallel(arr):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    arr_np = np.array(arr, dtype=np.int32)
    max_value = max(arr_np)
    temp_arr = np.zeros_like(arr_np)

    # Realizar la dispersión del arreglo local
    comm.Scatter([arr_np, MPI.INT], [temp_arr, MPI.INT], root=0)

    # Determinar el tamaño real del arreglo local
    local_size = len(temp_arr)

    # Crear un arreglo de conteo para contar las ocurrencias de cada elemento
    count = np.zeros(max_value + 1, dtype=np.int32)

    # Contar las ocurrencias de cada elemento en el arreglo local
    for num in temp_arr:
        count[num] += 1

    # Recolectar los conteos de cada proceso
    global_count = np.zeros_like(count)
    comm.Allreduce(count, global_count, op=MPI.SUM)

    # Reconstruir el arreglo ordenado utilizando el arreglo de conteo
    sorted_arr = np.zeros(sum(global_count), dtype=np.int32)
    offset = np.zeros(max_value + 1, dtype=np.int32)

    for i in range(1, max_value + 1):
        offset[i] = offset[i - 1] + global_count[i - 1]

    for i in range(local_size):
        sorted_arr[offset[temp_arr[i]]] = temp_arr[i]
        offset[temp_arr[i]] += 1

    return sorted_arr.tolist()

# Ejemplo de uso
if __name__ == "__main__":
    arr = [4, 2, 2, 8, 3, 3, 1]
    print("Arreglo original:", arr)

    sorted_arr = counting_sort_parallel(arr)
    print("Arreglo ordenado:", sorted_arr)



Arreglo original: [4, 2, 2, 8, 3, 3, 1]
Arreglo ordenado: [1, 2, 2, 3, 3, 4, 8]
